## Upsert in PySpark

In [ ]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Upsert") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

spark

In [ ]:
# Set the mode to dynamic to work on Upsert
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

# Check the mode for Partition Overwrite
print(spark.conf.get("spark.sql.sources.partitionOverwriteMode"))

In [ ]:
# Create the Full history dataset
from pyspark.sql.functions import to_date
_data = [
    ["ORD1001", "P003", 70, "01-21-2022", "01-30-2022"],
    ["ORD1004", "P033", 12, "01-24-2022", "01-30-2022"],
    ["ORD1005", "P036", 10, "01-20-2022", "01-30-2022"],
    ["ORD1002", "P016", 2, "01-10-2022", "01-30-2022"],
    ["ORD1003", "P012", 6, "01-10-2022", "01-30-2022"],
]

_cols = ["order_id", "prod_id", "qty", "order_date", "insert_date"]

# Create the dataframe
df = spark.createDataFrame(data=_data, schema=_cols)

# Cast the Order date from String to Date
df = df.withColumn("order_date", to_date("order_date" ,"MM-dd-yyyy")).withColumn("insert_date", to_date("insert_date" ,"MM-dd-yyyy"))
df.printSchema()
df.show()

In [ ]:
# Write the history data in as partitioned output partitioned by order_date
df.repartition("order_date") \
    .write \
    .format("parquet") \
    .partitionBy("order_date") \
    .mode("overwrite") \
    .save("dataset/orders_h")

In [ ]:
%%sh

ls -ltr dataset/orders_h/

In [ ]:
# Validate data
from pyspark.sql.functions import count, lit

spark.read.parquet("dataset/orders_h/").groupBy("order_date").agg(count(lit(1))).show()
df = spark.read.parquet("dataset/orders_h")
df.printSchema()
df.show()

In [ ]:
# Lets create our delta dataset for Upsert
# Consider the Order ORD1002 the qty is update to 16 and new order ORD1011 is added on 01-20-2022
_data = [
    ["ORD1002", "P016", 16, "01-10-2022", "01-31-2022"],
    ["ORD1011", "P076", 21, "01-20-2022", "01-31-2022"],
]

_cols = ["order_id", "prod_id", "qty", "order_date", "insert_date"]

# Create the delta dataframe
delta_df = spark.createDataFrame(data=_data, schema=_cols)

# Cast the Order date from String to Date
delta_df = delta_df.withColumn("order_date", to_date("order_date" ,"MM-dd-yyyy")).withColumn("insert_date", to_date("insert_date" ,"MM-dd-yyyy"))
delta_df.printSchema()
delta_df.show()

In [ ]:
# For Upsert identify the impacted partitions from delta data
impacted_df = delta_df.select("order_date").distinct()
impacted_df.show()

In [ ]:
# Select only the impacted partitions from history dataset
impacted_history_df = df.join(impacted_df, on="order_date", how="inner")
impacted_history_df.show()

In [ ]:
# Union with delta_df and find out the number records for impacted partitions
from pyspark.sql.functions import row_number, expr

union_df = impacted_history_df.unionByName(delta_df)
ranked_delta_df = union_df.withColumn("_rank", expr("row_number() over (partition by order_id order by insert_date desc)"))
ranked_delta_df.show()

In [ ]:
# Select the records based rank to upsert

final_delta_df = ranked_delta_df.where("_rank = 1")
final_delta_df.show()

In [ ]:
# Lets drop the rank column write the data into the history now

final_delta_df.drop("_rank").repartition("order_date") \
    .write \
    .format("parquet") \
    .partitionBy("order_date") \
    .mode("overwrite") \
    .save("dataset/orders_h")

In [ ]:
%%sh

ls -ltr dataset/orders_h/

In [ ]:
# Validate data
from pyspark.sql.functions import count, lit

spark.read.parquet("dataset/orders_h/").groupBy("order_date").agg(count(lit(1))).show()
df = spark.read.parquet("dataset/orders_h")
df.show()